In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import subprocess
import sys
import os
import glob
import json
# try:
#   import open3d as o3d
# except:
#   subprocess.check_call([sys.executable, "-m", "pip", "install", "open3d"])
#   import open3d as o3d
import open3d as o3d
from config import *
from utils import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 1. Read Points Cloud

In [22]:
# read ply file
# path = img_dir + "/1.ply"
path = "1.ply"
pcd = o3d.io.read_point_cloud(path)
# ply_file.paint_uniform_color([1, 0.706, 0])

# visualize ply file
o3d.visualization.draw_geometries([pcd])


In [3]:
n_xyz = np.asarray(pcd.points)
print(n_xyz)
print("----------------------------")
n_xyz[:, 0] > 0

[[-1.97852   1.19141  -2.0918  ]
 [-1.96973   1.19141  -2.0918  ]
 [-1.97461   1.20117  -2.10742 ]
 ...
 [ 0.442383 -0.252197 -0.442627]
 [ 0.44458  -0.252197 -0.442627]
 [ 0.446533 -0.252197 -0.442627]]
----------------------------


array([False, False, False, ...,  True,  True,  True])

# 2. Crop Img

In [5]:
#crop data
def crop(pcd, range_x, range_y, range_z):
  n_xyz = np.asarray(pcd.points)
  mask_x = (n_xyz[:, 0] > range_x[0]) & (n_xyz[:, 0] < range_x[1])
  
  # print(mask_x)
  mask_y = (n_xyz[:, 1] > range_y[0]) & (n_xyz[:, 1] < range_y[1])
  mask_z = (n_xyz[:, 2] > range_z[0]) & (n_xyz[:, 2] < range_z[1])
  mask = mask_x & mask_y & mask_z
  # print(mask)
  cropPCD = o3d.geometry.PointCloud()
  cropPCD.points = o3d.utility.Vector3dVector(n_xyz[mask])
  return cropPCD

newPCD = crop(pcd, [-0.5,0.5], [-0.5,0.5], [-1.25,0])
# o3d.visualization.draw_geometries([newPCD])
# crop2(pcd, min_x, min_y, min_z)

# 3. DownSampling

In [6]:
#down sample
downPCD = newPCD.voxel_down_sample(voxel_size = 0.01  )
# downPCD = voxel_down_sample(newPCD, voxel_size = 0.01) => error

#rotate img
# R = downPCD.get_rotation_matrix_from_xyz((0.7 * np.pi, 0, 0.6 * np.pi))
# downPCD = downPCD.rotate(R, center=(0,0,0))
# o3d.visualization.draw_geometries([downPCD])

# 4. Plane segmentation

In [8]:
# Plane segmentation
def plane(pcd):
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
    [a, b, c, d] = plane_model
    print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

    plane = pcd.select_by_index(inliers)
    # Count points in plane
    points_plane = np.asarray(plane.points)
    print(f"Number of points in the plane: {points_plane.shape[0]}")

    # inlier_cloud.paint_uniform_color([1.0, 0, 0])
    other = pcd.select_by_index(inliers, invert=True)
    points_others = np.asarray(other.points)
    print(f"Number of points in the plane: {points_others.shape[0]}")

    # Processing with loop
    i = 1
    while points_others.shape[0] > 0:
        print("Processing with the {} loop".format(i))
        i += 1
        print("Total surfaces:", i)
        # Segment plane
        plane_model, inliers = other.segment_plane(distance_threshold=0.01,
                                                    ransac_n=3,
                                                    num_iterations=1000)
        # Extract inliers and outliers
        plane = other.select_by_index(inliers)
        other = other.select_by_index(inliers, invert=True)
        # Count points in plane
        points_plane = np.asarray(plane.points)
        print(f"Number of points in the plane: {points_plane.shape[0]}")
        # Count points in other
        points_others = np.asarray(other.points)
        print(f"Number of points in the other: {points_others.shape[0]}")
        # Visualize
        o3d.visualization.draw_geometries([plane])
        # Break if no points left
        if points_others.shape[0] == 0:
            break
        # Update
        plane = other
        # other = other.select_by_index(inliers, invert=True)
        return plane


    # return plane, points_plane, other, points_others
plane = plane(downPCD)

# place_plane, count_points_1, other, points_others = plane(downPCD)
# o3d.visualization.draw_geometries([other])


Plane equation: -0.08x + 0.84y + 0.54z + 0.48 = 0
Number of points in the plane: 6091
Number of points in the plane: 1997
Processing with the 1 loop
Number of points in the plane: 1143
Number of points in the other: 854


# 5. Count the number of points in each plane

In [10]:
## License: Apache 2.0. See LICENSE file in root directory.
## Copyright(c) 2017 Intel Corporation. All Rights Reserved.

#####################################################
##          rs400 advanced mode tutorial           ##
#####################################################

# First import the library
import pyrealsense2 as rs
import time
import json

DS5_product_ids = ["0AD1", "0AD2", "0AD3", "0AD4", "0AD5", "0AF6", "0AFE", "0AFF", "0B00", "0B01", "0B03", "0B07", "0B3A", "0B5C"]

def find_device_that_supports_advanced_mode() :
    ctx = rs.context()
    ds5_dev = rs.device()
    devices = ctx.query_devices();
    for dev in devices:
        if dev.supports(rs.camera_info.product_id) and str(dev.get_info(rs.camera_info.product_id)) in DS5_product_ids:
            if dev.supports(rs.camera_info.name):
                print("Found device that supports advanced mode:", dev.get_info(rs.camera_info.name))
            return dev
    raise Exception("No D400 product line device that supports advanced mode was found")

try:
    dev = find_device_that_supports_advanced_mode()
    advnc_mode = rs.rs400_advanced_mode(dev)
    print("Advanced mode is", "enabled" if advnc_mode.is_enabled() else "disabled")

    # Loop until we successfully enable advanced mode
    while not advnc_mode.is_enabled():
        print("Trying to enable advanced mode...")
        advnc_mode.toggle_advanced_mode(True)
        # At this point the device will disconnect and re-connect.
        print("Sleeping for 5 seconds...")
        time.sleep(5)
        # The 'dev' object will become invalid and we need to initialize it again
        dev = find_device_that_supports_advanced_mode()
        advnc_mode = rs.rs400_advanced_mode(dev)
        print("Advanced mode is", "enabled" if advnc_mode.is_enabled() else "disabled")

    # Get each control's current value
    print("Depth Control: \n", advnc_mode.get_depth_control())
    print("RSM: \n", advnc_mode.get_rsm())
    print("RAU Support Vector Control: \n", advnc_mode.get_rau_support_vector_control())
    print("Color Control: \n", advnc_mode.get_color_control())
    print("RAU Thresholds Control: \n", advnc_mode.get_rau_thresholds_control())
    print("SLO Color Thresholds Control: \n", advnc_mode.get_slo_color_thresholds_control())
    print("SLO Penalty Control: \n", advnc_mode.get_slo_penalty_control())
    print("HDAD: \n", advnc_mode.get_hdad())
    print("Color Correction: \n", advnc_mode.get_color_correction())
    print("Depth Table: \n", advnc_mode.get_depth_table())
    print("Auto Exposure Control: \n", advnc_mode.get_ae_control())
    print("Census: \n", advnc_mode.get_census())

    #To get the minimum and maximum value of each control use the mode value:
    query_min_values_mode = 1
    query_max_values_mode = 2
    current_std_depth_control_group = advnc_mode.get_depth_control()
    min_std_depth_control_group = advnc_mode.get_depth_control(query_min_values_mode)
    max_std_depth_control_group = advnc_mode.get_depth_control(query_max_values_mode)
    print("Depth Control Min Values: \n ", min_std_depth_control_group)
    print("Depth Control Max Values: \n ", max_std_depth_control_group)

    # Set some control with a new (median) value
    current_std_depth_control_group.scoreThreshA = int((max_std_depth_control_group.scoreThreshA - min_std_depth_control_group.scoreThreshA) / 2)
    advnc_mode.set_depth_control(current_std_depth_control_group)
    print("After Setting new value, Depth Control: \n", advnc_mode.get_depth_control())

    # Serialize all controls to a Json string
    serialized_string = advnc_mode.serialize_json()
    print("Controls as JSON: \n", serialized_string)
    as_json_object = json.loads(serialized_string)

    # We can also load controls from a json string
    # For Python 2, the values in 'as_json_object' dict need to be converted from unicode object to utf-8
    if type(next(iter(as_json_object))) != str:
        as_json_object = {k.encode('utf-8'): v.encode("utf-8") for k, v in as_json_object.items()}
    # The C++ JSON parser requires double-quotes for the json object so we need
    # to replace the single quote of the pythonic json to double-quotes
    json_string = str(as_json_object).replace("'", '\"')
    advnc_mode.load_json(json_string)

except Exception as e:
    print(e)
    pass
1

Found device that supports advanced mode: Intel RealSense D435
Advanced mode is enabled
Depth Control: 
 minusDecrement: 10, deepSeaMedianThreshold: 500, scoreThreshA: 511, scoreThreshB: 2047, textureDifferenceThreshold: 0, textureCountThreshold: 0, deepSeaSecondPeakThreshold: 325, deepSeaNeighborThreshold: 7, lrAgreeThreshold: 24
RSM: 
 rsmBypass: 0, diffThresh: 4, sloRauDiffThresh: 1, removeThresh: 63
RAU Support Vector Control: 
 minWest: 1, minEast: 1, minWEsum: 3, minNorth: 1, minSouth: 1, minNSsum: 3, uShrink: 3, vShrink: 1
Color Control: 
 disableSADColor: 0, disableRAUColor: 0, disableSLORightColor: 0, disableSLOLeftColor: 0, disableSADNormalize: 0
RAU Thresholds Control: 
 rauDiffThresholdRed: 51, rauDiffThresholdGreen: 51, rauDiffThresholdBlue: 51
SLO Color Thresholds Control: 
 diffThresholdRed: 72, diffThresholdGreen: 72, diffThresholdBlue: 72
SLO Penalty Control: 
 sloK1Penalty: 60, sloK2Penalty: 342, sloK1PenaltyMod1: 105, sloK2PenaltyMod1: 190, sloK1PenaltyMod2: 70, sloK

1

# Read video

In [16]:
## License: Apache 2.0. See LICENSE file in root directory.
## Copyright(c) 2015-2017 Intel Corporation. All Rights Reserved.

###############################################
##      Open CV and Numpy integration        ##
###############################################

import pyrealsense2 as rs
import numpy as np
import cv2

from config import *

# Create Trackbar for Threshold
# cv2.namedWindow('Depth Image')
cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
cv2.createTrackbar('FPS', 'RealSense', 0, 50, lambda x: x+1)

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

fps = cv2.getTrackbarPos('FPS', 'RealSense')
print(fps)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, fps)

if device_product_line == 'L500':
    config.enable_stream(rs.stream.color, 960, 540, rs.format.bgr8, fps)
else:
    config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, fps)

# Start streaming
pipeline.start(config)

try:
    while True:
        # print(pipeline)
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue
        
        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            resized_color_image = cv2.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            images = np.hstack((resized_color_image, depth_colormap))
        else:
            images = np.hstack((color_image, depth_colormap))
        
        # Flip camera image
        images = cv2.flip(images, 1)

        # Save images
        import os
        # Create directory if not exist
        if not os.path.exists('images'):
            os.mkdir('images')
        
        if count // 2 == 0:
            if len(os.listdir("./images"))!=10:
                cv2.imwrite("./images/image_" + str(count) + ".png", images)    
        count += 1

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        key = cv2.waitKey(1)
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
        
finally:

    # Stop streaming
    pipeline.stop()

0


In [21]:
import pyrealsense2 as rs
import numpy as np

fps = 30

def initialize_camera():
    # start the frames pipe
    p = rs.pipeline()
    conf = rs.config()
    # conf.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, fps)
    conf.enable_stream(rs.stream.accel)
    conf.enable_stream(rs.stream.gyro)
    # conf.enable_stream(rs.stream.pose)
    prof = p.start(conf)
    return prof #????


def gyro_data(gyro):
    return np.asarray([gyro.x, gyro.y, gyro.z])


def accel_data(accel):
    return np.asarray([accel.x, accel.y, accel.z])


def pose_data(pose):
    return np.asarray([pose.tx, pose.ty, pose.tz, pose.qx, pose.qy, pose.qz, pose.qw])

p = initialize_camera()
try:
    while True:
        f = p.wait_for_frames()
        accel = accel_data(f[0].as_motion_frame().get_motion_data())
        gyro = gyro_data(f[1].as_motion_frame().get_motion_data())
        # pose = pose_data(f[2].as_pose_frame().get_pose_data())
        print("accelerometer: ", accel)
        print("gyro: ", gyro)

finally:
    p.stop()

RuntimeError: Failed to open scan_element /sys/devices/pci0000:00/0000:00:14.0/usb2/2-4/2-4:1.5/0003:8086:0B5C.0005/HID-SENSOR-200073.3.auto/iio:device1/scan_elements/in_accel_y_en Last Error: Permission denied